In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, RNN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [2]:
# Đọc dữ liệu từ file CSV (có thể thay bằng file của bạn)
data = pd.read_csv('D:\DeTaiNam2024\SoLieuHong\VN30.csv')
# Kiểm tra tiêu đề các cột
data.head()

,Unnamed: 0,time,open,high,low,close,volume
0,0,2017-08-24,743.23,749.82,742.34,747.72,33630890
1,1,2017-08-25,746.87,751.14,746.87,751.14,32784240
2,2,2017-08-28,751.59,758.02,749.20,758.02,42763860
3,3,2017-08-29,757.84,758.76,754.59,754.59,44549640
4,4,2017-08-30,756.49,764.57,755.32,764.57,34232490


In [3]:
 data = data[['close']]
 data.dropna(inplace=True)

In [4]:
# Chuẩn bị dữ liệu
def prepare_data(data, time_steps=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(time_steps, len(scaled_data)):
        X.append(scaled_data[i-time_steps:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)

    # Reshape X để phù hợp với RNN [samples, time_steps, features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y, scaler


In [5]:
X, y, scaler = prepare_data(data)

In [6]:
class sLSTMCell(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(sLSTMCell, self).__init__(**kwargs)
        self.units = units
        self.state_size = [self.units, self.units]  # [hidden_state, normalizer_state]

    def build(self, input_shape):
        # Khởi tạo trọng số cho các cổng
        self.W_f = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='W_f')
        self.U_f = self.add_weight(shape=(self.units, self.units),
                                   initializer='orthogonal',
                                   name='U_f')
        self.b_f = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_f')

        self.W_i = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='W_i')
        self.U_i = self.add_weight(shape=(self.units, self.units),
                                   initializer='orthogonal',
                                   name='U_i')
        self.b_i = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_i')

        self.W_c = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='W_c')
        self.U_c = self.add_weight(shape=(self.units, self.units),
                                   initializer='orthogonal',
                                   name='U_c')
        self.b_c = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_c')

        self.W_o = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='W_o')
        self.U_o = self.add_weight(shape=(self.units, self.units),
                                   initializer='orthogonal',
                                   name='U_o')
        self.b_o = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_o')
        super(sLSTMCell, self).build(input_shape)

    def call(self, inputs, states):
        h_prev, n_prev = states

        # Forget gate: sử dụng sigmoid hoặc exp
       # f = tf.keras.activations.sigmoid(tf.matmul(inputs, self.W_f) + tf.matmul(h_prev, self.U_f) + self.b_f)
        f = tf.exp(tf.matmul(inputs, self.W_f) + tf.matmul(h_prev, self.U_f) + self.b_f)

        # Input gate: sử dụng (sigmoid)^2 hoặc (exp)^2
      #  i = tf.keras.activations.sigmoid(tf.matmul(inputs, self.W_i) + tf.matmul(h_prev, self.U_i) + self.b_i)
       # i = tf.square(i)  # (σ(x))^2
        i = tf.exp(tf.matmul(inputs, self.W_i) + tf.matmul(h_prev, self.U_i) + self.b_i)
        # Candidate cell state
        c_tilde = tf.keras.activations.tanh(tf.matmul(inputs, self.W_c) + tf.matmul(h_prev, self.U_c) + self.b_c)

        # Cell state update
        c = f * states[0] + i * c_tilde

        # Normalizer state update
        n = f * n_prev + i

        # Output gate
        o = tf.keras.activations.sigmoid(tf.matmul(inputs, self.W_o) + tf.matmul(h_prev, self.U_o) + self.b_o)

        # Normalized hidden state
        h_tilde = c * tf.math.reciprocal(n + 1e-7)  # thêm epsilon để tránh chia cho 0
        h = o * h_tilde

        return h, [h, n]

class sLSTM(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(sLSTM, self).__init__(**kwargs)
        self.units = units
        self.cell = sLSTMCell(units)

    def call(self, inputs):
        return tf.keras.layers.RNN(self.cell, return_sequences=False, return_state=False)(inputs)

    def get_config(self):
        config = super(sLSTM, self).get_config()
        config.update({'units': self.units})
        return config


In [12]:
# Xây dựng mô hình
def build_model(time_steps, units=50):
    inputs = Input(shape=(time_steps, 1))
    x = sLSTM(units)(inputs)
    x = Dense(1)(x)
    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate=2e-06), loss='mean_squared_error')
    return model
# Khởi tạo mô hình
model = build_model(time_steps=60, units=50)
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 60, 1)]           0         
                                                                 
 s_lstm_3 (sLSTM)            (None, 50)                10400     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Xây dựng mô hình
model = build_model(time_steps=60, units=50)

# Huấn luyện mô hình
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop])


Epoch 1/100
42/42 [==============================] - 21s 117ms/step - loss: 0.2928 - val_loss: 0.3403
Epoch 2/100
42/42 [==============================] - 3s 75ms/step - loss: 0.2927 - val_loss: 0.3401
Epoch 3/100
42/42 [==============================] - 3s 80ms/step - loss: 0.2925 - val_loss: 0.3399
Epoch 4/100
42/42 [==============================] - 3s 81ms/step - loss: 0.2923 - val_loss: 0.3397
Epoch 5/100
42/42 [==============================] - 3s 82ms/step - loss: 0.2922 - val_loss: 0.3395
Epoch 6/100
42/42 [==============================] - 3s 74ms/step - loss: 0.2920 - val_loss: 0.3394
Epoch 7/100
42/42 [==============================] - 3s 76ms/step - loss: 0.2918 - val_loss: 0.3392
Epoch 8/100
42/42 [==============================] - 3s 74ms/step - loss: 0.2917 - val_loss: 0.3390
Epoch 9/100
42/42 [==============================] - 3s 72ms/step - loss: 0.2915 - val_loss: 0.3388
Epoch 10/100
42/42 [==============================] - 3s 79ms/step - loss: 0.2913 - val_loss: 0.33

In [11]:

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đảo chuẩn hóa để lấy giá trị thực
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Tính toán RMSE
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
print(f'Root Mean Squared Error (RMSE): {rmse}')
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f'Mean Squared Error (MSE): {mse}')
# Vẽ đồ thị so sánh giá thực và giá dự đoán
plt.figure(figsize=(14, 5))
plt.plot(y_test_inv, color='blue', label='Giá Thực')
plt.plot(y_pred_inv, color='red', label='Giá Dự Đoán')
plt.title('Dự Đoán Giá Cổ Phiếu VN30 Sử Dụng sLSTM')
plt.xlabel('Thời Gian')
plt.ylabel('Giá Cổ Phiếu')
plt.legend()
plt.show()


11/11 [==============================] - 2s 13ms/step


ValueError: Input contains NaN.

In [ ]:
mse1=np.mean(np.square(y_pred_inv - y_test_inv))
print('MSE=', mse1)
mean_error_percent1 = np.mean(np.abs((y_pred_inv - y_test_inv) / y_test_inv))
accuracy1 = 1 - mean_error_percent1
print('Accuracy=', accuracy1)
print('mean_error_percent=', mean_error_percent1)